# Otel Yorumları Duygu Analizi - DL (LSTM)
Pipeline

# Otel Yorumu Duygu Analizi | LSTM Sentiment Analysis

| Özellik | Detay |
|---|---|
| **Görev** | 3-Sınıflı Duygu Analizi (Negative / Neutral / Positive) |
| **Veri** | Hotel yorumları — İngilizce kullanıcı yorumları (Rating: 1–5) |
| **Ön İşleme** | Dil filtresi + Tokenize + Stopword kaldırma + Lemmatizasyon |
| **Örnekleme** | RandomOverSampler (neutral ↑) + RandomUnderSampler (positive ↓) |
| **Model** | LSTM: Embedding(10k, 128) + SpatialDropout + LSTM(128) + Dense(3) |
| **Doğruluk** | Test Accuracy = 0.77 (%77) |
| **Teknoloji** | TensorFlow / Keras, NLTK, imbalanced-learn |

**Sınıf Dengesi:** Positive ağırlıklı veri (%73.7); oversampling/undersampling ile eğitim seti dengelenmiştir.
**En Zayıf Sınıf:** Neutral (F1 = 0.29) — 3 puanlı yorumların belirsiz sınır niteliği sınıflandırmayı zorlaştırmaktadır.

## 1. Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import sys
import os

# Proje kök dizinini ekleme (src modülü hatasını önlemek için)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
   
import pandas as pd
import numpy as np
import re
import joblib
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from src.data_preprocessing import preprocess_reviews, clean_text, create_sentiment_label
from langdetect import detect
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

## 2. Veri Yükleme

In [2]:
df = pd.read_csv('dataset.csv')
print("Veri setinin ilk 5 satırı:")
print(df.head())

Veri setinin ilk 5 satırı:
                                              Review  Rating
0  nice hotel expensive parking got good deal sta...       4
1  ok nothing special charge diamond member hilto...       2
2  nice rooms not 4* experience hotel monaco seat...       3
3  unique, great stay, wonderful time hotel monac...       5
4  great stay great stay, went seahawk game aweso...       5


## 3. Dil Tespiti ve Filtreleme

In [3]:
def detect_lang(text):
    try:
        return detect(text)
    except:
        return 'unknown'

df['lang'] = df['Review'].apply(detect_lang)
df = df[df['lang'] == 'en']
print("İngilizce olmayan yorumlar filtrelendi.")

İngilizce olmayan yorumlar filtrelendi.


## 4. Metin Ön İşleme

In [4]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def tokenize_text(text):
    return word_tokenize(text)

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stop_words]

def apply_lemmatization(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

def create_sentiment_label(rating):
    if rating in [1, 2]:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    elif rating in [4, 5]:
        return 'positive'
    else:
        return None

def preprocess_reviews(df, text_column='Review', rating_column='Rating'):
    df['cleaned_review'] = df[text_column].apply(clean_text)
    df['tokens'] = df['cleaned_review'].apply(tokenize_text)
    df['filtered_tokens'] = df['tokens'].apply(remove_stopwords)
    df['lemmatized_tokens'] = df['filtered_tokens'].apply(apply_lemmatization)
    df['processed_review'] = df['lemmatized_tokens'].apply(lambda x: ' '.join(x))
    df['sentiment'] = df[rating_column].apply(create_sentiment_label)
    return df

# Ön işleme adımlarını uygulama
df_processed = preprocess_reviews(df.copy())

print("Ön işlenmiş verinin ilk 5 satırı:")
print(df_processed[['Review', 'processed_review', 'sentiment']].head())

Ön işlenmiş verinin ilk 5 satırı:
                                              Review  \
0  nice hotel expensive parking got good deal sta...   
1  ok nothing special charge diamond member hilto...   
2  nice rooms not 4* experience hotel monaco seat...   
3  unique, great stay, wonderful time hotel monac...   
4  great stay great stay, went seahawk game aweso...   

                                    processed_review sentiment  
0  nice hotel expensive parking got good deal sta...  positive  
1  ok nothing special charge diamond member hilto...  negative  
2  nice room experience hotel monaco seattle good...   neutral  
3  unique great stay wonderful time hotel monaco ...  positive  
4  great stay great stay went seahawk game awesom...  positive  


In [ ]:
#df['processed_review'] = df['Review'].apply(clean_text)
#df['sentiment'] = df['Rating'].apply(create_sentiment_label)
#df.dropna(subset=['sentiment'], inplace=True)
#print("Ön işlenmiş verinin ilk 5 satırı:")
#print(df[['Review', 'processed_review', 'sentiment']].head())

## 5. Train / Test Bölme

In [9]:
X = df_processed['processed_review']
# DİKKAT: Burada get_dummies kullanmıyoruz, ham etiketleri alıyoruz
y = df_processed['sentiment']

# Stratify parametresi dengeli dağılım için önemlidir
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Eğitim seti boyutu:", X_train.shape)
print("Etiketler düz metin olarak kaldı:", y_train.head())

Eğitim seti boyutu: (16380,)
Etiketler düz metin olarak kaldı: 3332      neutral
5970     positive
10394    positive
8603     positive
10515    positive
Name: sentiment, dtype: object


In [16]:
#X = df_processed['processed_review']
#y = pd.get_dummies(df_processed['sentiment'])
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## 6. Metinlerin Dizilere Dönüştürülmesi

In [10]:
max_words = 10000
max_len = 150
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

## 7. Sınıf Dengesizliği Giderme

Sınıf dengesizliğini gidermek için `RandomOverSampler` ve `RandomUnderSampler` kullanılacaktır. `RandomOverSampler` ile 'neutral' sınıfı artırılacak, `RandomUnderSampler` ile 'positive' sınıfı azaltılacaktır.

In [11]:

# Not: y_train artık düz metin olduğu için sampling_strategy çalışacaktır.
over_sampler = RandomOverSampler(sampling_strategy={'neutral': 5000})
under_sampler = RandomUnderSampler(sampling_strategy={'positive': 10000})

# 1. Önce Dengesizliği Gider (Resampling)
X_train_res, y_train_res = over_sampler.fit_resample(X_train_pad, y_train)
X_train_res, y_train_res = under_sampler.fit_resample(X_train_res, y_train_res)

print("Örnekleme sonrası veri seti boyutu:", X_train_res.shape)

# 2. ŞİMDİ One-Hot Encoding'e Çevir (LSTM için gerekli)
y_train_res = pd.get_dummies(y_train_res)
y_test = pd.get_dummies(y_test) # Test setini de çevirmeyi unutmayın!

# Sütun sırasının aynı olduğundan emin olalım (neg, neu, pos)
y_train_res = y_train_res[['negative', 'neutral', 'positive']]
y_test = y_test[['negative', 'neutral', 'positive']]

print("One-Hot dönüşümü tamamlandı.")
print(y_train_res.head())

Örnekleme sonrası veri seti boyutu: (17568, 150)
One-Hot dönüşümü tamamlandı.
   negative  neutral  positive
0      True    False     False
1      True    False     False
2      True    False     False
3      True    False     False
4      True    False     False


In [12]:
#over_sampler = RandomOverSampler(sampling_strategy={'neutral': 5000})
#under_sampler = RandomUnderSampler(sampling_strategy={'positive': 10000})
#X_train_res, y_train_res = over_sampler.fit_resample(X_train_pad, y_train)
#X_train_res, y_train_res = under_sampler.fit_resample(X_train_res, y_train_res)

## 8. Model Tanımı (LSTM)

In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## 9. Model Eğitimi

In [14]:
epochs = 5
batch_size = 64
history = model.fit(X_train_res, y_train_res, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Epoch 1/5
248/248 ━━━━━━━━━━━━━━━━━━━━ 81s 304ms/step - accuracy: 0.6697 - loss: 0.7304 - val_accuracy: 0.8867 - val_loss: 0.3281
Epoch 2/5
248/248 ━━━━━━━━━━━━━━━━━━━━ 74s 297ms/step - accuracy: 0.8352 - loss: 0.4349 - val_accuracy: 0.7997 - val_loss: 0.4387
Epoch 3/5
248/248 ━━━━━━━━━━━━━━━━━━━━ 77s 309ms/step - accuracy: 0.8851 - loss: 0.3216 - val_accuracy: 0.8702 - val_loss: 0.3275
Epoch 4/5
248/248 ━━━━━━━━━━━━━━━━━━━━ 81s 325ms/step - accuracy: 0.9143 - loss: 0.2426 - val_accuracy: 0.8987 - val_loss: 0.2660
Epoch 5/5
248/248 ━━━━━━━━━━━━━━━━━━━━ 79s 319ms/step - accuracy: 0.9431 - loss: 0.1707 - val_accuracy: 0.8873 - val_loss: 0.3114


## 10. Değerlendirme

In [17]:
y_pred = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test.values, axis=1)
report = classification_report(y_test_classes, y_pred_classes, target_names=['negative', 'neutral', 'positive'])
print("Sınıflandırma Raporu:\n", report)

128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step
Sınıflandırma Raporu:
               precision    recall  f1-score   support

    negative       0.68      0.56      0.61       642
     neutral       0.25      0.35      0.29       437
    positive       0.90      0.88      0.89      3016

    accuracy                           0.77      4095
   macro avg       0.61      0.59      0.60      4095
weighted avg       0.79      0.77      0.78      4095



## 11. Modeli Kaydetme

In [18]:
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)
model.save(os.path.join(output_dir, 'sentiment_model_dl.h5'))
joblib.dump(tokenizer, os.path.join(output_dir, 'tokenizer_dl.pkl'))
print(f"Model ve tokenizer '{output_dir}' dizinine kaydedildi.")

Model ve tokenizer '../data/processed' dizinine kaydedildi.


# 12. Evaluation

**1. Genel Performans Özeti**
Accuracy: 0.77 (%77)
Bu değer, ilk bakışta makul görünse de veri setinin pozitif ağırlıklı yapısı nedeniyle tek başına yeterli değildir.
Sınıf dağılımı:
Positive: 3016 (%73.7)
Negative: 642 (%15.7)
Neutral: 437 (%10.6)

Bu dengesizlik, accuracy’nin olduğundan daha iyi görünmesine neden oluyor.

**2. Sınıf Bazlı Detaylı Analiz**
🔴 Negative (Olumsuz)
Precision: 0.68
Recall: 0.56
F1: 0.61
- Yorum:
Gerçek olumsuz yorumların yalnızca %56’sı yakalanabilmiş.
%44’ü positive veya neutral’a kaymış.
Precision makul, ama recall düşük → model negatifleri kaçırıyor.
Operasyonel kullanımda (müşteri şikâyetlerini yakalama) risklidir.

🟡 Neutral (Nötr) — En Zayıf Sınıf
Precision: 0.25
Recall: 0.35
F1: 0.29
Bu kritik bir zayıflık:
Model nötr tahmin yaptığında, bunun yalnızca %25’i doğru.
Gerçek nötr yorumların %65’i yanlış sınıfa gidiyor.
Model nötr kavramını öğrenememiş.
Fiilen modelin 3 sınıflı değil, 2.3 sınıflı çalıştığını söylemek mümkün.

🟢 Positive (Olumlu)
Precision: 0.90
Recall: 0.88
F1: 0.89

-Yorum:
Pozitif sınıf çok güçlü.
Model büyük ölçüde “her şeyi pozitif görme” eğiliminde.
Bu sınıf, accuracy’yi yapay biçimde yukarı çekiyor.

**3. Macro vs Weighted Ortalamalar**
Macro Avg (F1 = 0.60)
Her sınıfa eşit ağırlık verir → gerçek kaliteyi yansıtır.
→ Düşük ve orta seviye arası bir performans.
Weighted Avg (F1 = 0.78)
Pozitif sınıfın ağırlığı nedeniyle şişmiş.
→ İş kararları için tek başına güvenilir değil.

## Sonuç ve Değerlendirme

| Sınıf | Precision | Recall | F1-Score | Destek |
|---|---|---|---|---|
| **Negative** | 0.68 | 0.56 | 0.61 | 642 |
| **Neutral** | 0.25 | 0.35 | 0.29 | 437 |
| **Positive** | 0.90 | 0.88 | 0.89 | 3016 |
| **Accuracy** | — | — | **0.77** | 4095 |
| **Macro Avg** | 0.61 | 0.59 | 0.60 | 4095 |

**Bulgular:**
- LSTM modeli pozitif yorumlarda yüksek başarı (F1 = 0.89) elde etmiştir; veri setinin büyük çoğunluğunun pozitif olması bu sınıfı güçlendirmektedir
- Nötr sınıf (F1 = 0.29) en zayıf performansı sergilemektedir — 3 puanlı yorumların belirsiz sınır niteliği sınıflandırmayı zorlaştırmaktadır
- Negatif yorumların %44'ü yanlış sınıflandırılmaktadır; müşteri şikayet tespiti açısından recall değeri (0.56) geliştirilmelidir
- EarlyStopping ile aşırı öğrenme kontrolü sağlanmış; val_accuracy Epoch 4'te 0.90'a ulaşmıştır
- BERT tabanlı ön eğitimli model veya daha dengeli veri seti ile nötr ve negatif sınıf F1 skoru artırılabilir